# PeopleRX: Drug Recommendation System (Part 2)

# import in libraries

In [1]:
import nltk
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import re, string
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# read in cleaned data

df_clean = pd.read_csv("C:/Users/New User/Desktop/NLP/drugsCom_raw/drugs_clean_data.csv")
df_clean.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,strlen,tokens,text
0,0,Valsartan,Left Ventricular Dysfunction,it has no side effect i take it in combination...,9.0,"May 20, 2012",27,16,"['it', 'has', 'no', 'side', 'effect', 'i', 'ta...",it has no side effect i take it in combination...
1,1,Guanfacine,ADHD,my son is halfway through his fourth week of i...,8.0,"April 27, 2010",192,140,"['my', 'son', 'is', 'halfway', 'through', 'his...",my son is halfway through his fourth week of i...
2,2,Lybrel,Birth Control,i used to take another oral contraceptive whic...,5.0,"December 14, 2009",17,132,"['i', 'used', 'to', 'take', 'another', 'oral',...",i used to take another oral contraceptive whic...
3,3,Ortho Evra,Birth Control,this is my first time using any form of birth ...,8.0,"November 3, 2015",10,86,"['this', 'is', 'my', 'first', 'time', 'using',...",this is my first time using any form of birth ...
4,4,Buprenorphine / naloxone,Opiate Dependence,suboxone has completely turned my life around ...,9.0,"November 27, 2016",37,125,"['suboxone', 'has', 'completely', 'turned', 'm...",suboxone has completely turned my life around ...


In [3]:
df_clean = df_clean.drop(['Unnamed: 0'], axis=1)
df_clean

,drugName,condition,review,rating,date,usefulCount,strlen,tokens,text
0,Valsartan,Left Ventricular Dysfunction,it has no side effect i take it in combination...,9.0,"May 20, 2012",27,16,"['it', 'has', 'no', 'side', 'effect', 'i', 'ta...",it has no side effect i take it in combination...
1,Guanfacine,ADHD,my son is halfway through his fourth week of i...,8.0,"April 27, 2010",192,140,"['my', 'son', 'is', 'halfway', 'through', 'his...",my son is halfway through his fourth week of i...
2,Lybrel,Birth Control,i used to take another oral contraceptive whic...,5.0,"December 14, 2009",17,132,"['i', 'used', 'to', 'take', 'another', 'oral',...",i used to take another oral contraceptive whic...
3,Ortho Evra,Birth Control,this is my first time using any form of birth ...,8.0,"November 3, 2015",10,86,"['this', 'is', 'my', 'first', 'time', 'using',...",this is my first time using any form of birth ...
4,Buprenorphine / naloxone,Opiate Dependence,suboxone has completely turned my life around ...,9.0,"November 27, 2016",37,125,"['suboxone', 'has', 'completely', 'turned', 'm...",suboxone has completely turned my life around ...
...,...,...,...,...,...,...,...,...,...
212845,Tamoxifen,"Breast Cancer, Prevention",i have taken tamoxifen for years side effects ...,10.0,"September 13, 2014",43,93,"['i', 'have', 'taken', 'tamoxifen', 'for', 'ye...",i have taken tamoxifen for years side effects ...
212846,Escitalopram,Anxiety,ive been taking lexapro escitaploprgram since ...,9.0,"October 8, 2016",11,132,"['ive', 'been', 'taking', 'lexapro', 'escitapl...",ive been taking lexapro escitaploprgram since ...
212847,Levonorgestrel,Birth Control,im married years old and i have no kids taking...,8.0,"November 15, 2010",7,146,"['im', 'married', 'years', 'old', 'and', 'i', ...",im married years old and i have no kids taking...
212848,Tapentadol,Pain,i was prescribed nucynta for severe neckshould...,1.0,"November 28, 2011",20,33,"['i', 'was', 'prescribed', 'nucynta', 'for', '...",i was prescribed nucynta for severe neckshould...


In [4]:
df_clean = df_clean
df_clean

,drugName,condition,review,rating,date,usefulCount,strlen,tokens,text
0,Valsartan,Left Ventricular Dysfunction,it has no side effect i take it in combination...,9.0,"May 20, 2012",27,16,"['it', 'has', 'no', 'side', 'effect', 'i', 'ta...",it has no side effect i take it in combination...
1,Guanfacine,ADHD,my son is halfway through his fourth week of i...,8.0,"April 27, 2010",192,140,"['my', 'son', 'is', 'halfway', 'through', 'his...",my son is halfway through his fourth week of i...
2,Lybrel,Birth Control,i used to take another oral contraceptive whic...,5.0,"December 14, 2009",17,132,"['i', 'used', 'to', 'take', 'another', 'oral',...",i used to take another oral contraceptive whic...
3,Ortho Evra,Birth Control,this is my first time using any form of birth ...,8.0,"November 3, 2015",10,86,"['this', 'is', 'my', 'first', 'time', 'using',...",this is my first time using any form of birth ...
4,Buprenorphine / naloxone,Opiate Dependence,suboxone has completely turned my life around ...,9.0,"November 27, 2016",37,125,"['suboxone', 'has', 'completely', 'turned', 'm...",suboxone has completely turned my life around ...
...,...,...,...,...,...,...,...,...,...
212845,Tamoxifen,"Breast Cancer, Prevention",i have taken tamoxifen for years side effects ...,10.0,"September 13, 2014",43,93,"['i', 'have', 'taken', 'tamoxifen', 'for', 'ye...",i have taken tamoxifen for years side effects ...
212846,Escitalopram,Anxiety,ive been taking lexapro escitaploprgram since ...,9.0,"October 8, 2016",11,132,"['ive', 'been', 'taking', 'lexapro', 'escitapl...",ive been taking lexapro escitaploprgram since ...
212847,Levonorgestrel,Birth Control,im married years old and i have no kids taking...,8.0,"November 15, 2010",7,146,"['im', 'married', 'years', 'old', 'and', 'i', ...",im married years old and i have no kids taking...
212848,Tapentadol,Pain,i was prescribed nucynta for severe neckshould...,1.0,"November 28, 2011",20,33,"['i', 'was', 'prescribed', 'nucynta', 'for', '...",i was prescribed nucynta for severe neckshould...


In [5]:
from nltk.corpus import stopwords

In [6]:
# adding new words to stopword list

stop_words = nltk.corpus.stopwords.words('english')
newStopWords = ['like','just', 'ive', 'im', 'got', 'day', 'days', 'year', 'years', 'week', 'weeks', 'took', 'taking', 'taken', 'side effects', 'months', 'mg', 'also', 'would', 'yet', 'side', 'effects', 'first', 'take', 'pill', 'medication', 'time', 'get', 'month', 'went', 'ago', 'feel', 'felt', 'one', 'hours', 'using', 'started', 'dont', 'since', 'life', 'never', 'really', 'medicine', 'dose', 'much', 'still', 'doctor', 'dr', 'meds', 'back']
stop_words.extend(newStopWords)
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
from nltk.corpus import stopwords

In [8]:
# creating document-term matrix with new stopwords

cv = CountVectorizer(stop_words=stop_words, max_df = .75, min_df=.02, ngram_range=(1,3))
X = cv.fit_transform(df_clean.review)
pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())

,able,absolutely,acne,actually,almost,along,already,although,always,amazing,...,wish,within,without,work,worked,working,works,worse,worst,worth
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212845,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212846,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
212847,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
212848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X.shape

(212850, 304)

In [10]:
# We have to convert `.toarray()` because the vectorizer returns a sparse matrix.
# For a big corpus, we would skip the dataframe and keep the output sparse.
pd.DataFrame(X.toarray(), index=df_clean.review, columns=cv.get_feature_names_out()).reset_index().head(10)

,review,able,absolutely,acne,actually,almost,along,already,although,always,...,wish,within,without,work,worked,working,works,worse,worst,worth
0,it has no side effect i take it in combination...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,my son is halfway through his fourth week of i...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,i used to take another oral contraceptive whic...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,this is my first time using any form of birth ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,suboxone has completely turned my life around ...,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
5,nd day on mg started to work with rock hard er...,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,1
6,he pulled out but he cummed a bit in me i took...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,abilify changed my life there is hope i was on...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,i ve had nothing but problems with the kepper...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,i had been on the pill for many years when my ...,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(6)
doc_topic = lsa.fit_transform(X)
lsa.explained_variance_ratio_

array([0.02295136, 0.03296352, 0.02429437, 0.01684869, 0.01622343,
       0.01428794])

In [12]:
topic_word = pd.DataFrame(lsa.components_.round(3),
#              index = ["component_1","component_2"],
             columns = cv.get_feature_names_out())
topic_word

,able,absolutely,acne,actually,almost,along,already,although,always,amazing,...,wish,within,without,work,worked,working,works,worse,worst,worth
0,0.048,0.023,0.114,0.036,0.085,0.019,0.026,0.020,0.058,0.024,...,0.016,0.051,0.056,0.114,0.074,0.041,0.074,0.061,0.043,0.036
1,0.026,-0.007,-0.118,-0.003,-0.006,0.008,-0.003,-0.000,-0.020,0.001,...,0.004,0.013,0.027,0.049,0.023,0.012,0.033,0.006,-0.003,0.002
2,-0.039,-0.000,0.036,-0.016,-0.017,-0.009,-0.016,-0.007,-0.017,-0.015,...,-0.009,-0.028,-0.038,-0.113,-0.060,-0.040,-0.054,-0.025,0.006,-0.010
3,0.003,-0.004,-0.083,0.006,-0.007,-0.002,0.005,0.000,0.001,-0.003,...,-0.003,-0.018,-0.007,-0.034,-0.030,0.009,-0.029,-0.047,-0.021,-0.003
4,0.013,0.004,-0.008,0.002,-0.020,0.005,-0.003,-0.001,-0.001,0.003,...,0.001,-0.016,0.007,-0.006,-0.007,0.001,-0.006,-0.006,-0.002,-0.005
5,-0.007,0.005,0.479,0.004,0.017,0.006,0.009,0.002,0.007,0.016,...,0.005,0.023,-0.002,0.026,0.046,0.012,0.024,0.048,0.007,0.019


In [13]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [14]:
display_topics(lsa, cv.get_feature_names_out(), 5)


Topic  0
pain, period, weight, control, bad

Topic  1
pain, sleep, severe, night, relief

Topic  2
pain, period, birth, control, birth control

Topic  3
weight, gain, lbs, weight gain, lost

Topic  4
control, birth, birth control, anxiety, depression

Topic  5
acne, skin, face, use, clear


In [15]:
Vt = pd.DataFrame(doc_topic.round(3),
             index = df_clean.review,
             columns = ['period', 'pain', 'birth control', 'weight', 'mood', 'acne'])
Vt

,period,pain,birth control,weight,mood,acne
review,,,,,,
it has no side effect i take it in combination of bystolic mg and fish oil,0.053,0.009,-0.041,0.006,0.012,0.000
my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective,1.810,0.084,-0.810,-0.278,-0.291,0.385
i used to take another oral contraceptive which had pill cycle and was very happy very light periods max days no other side effects but it contained hormone gestodene which is not available in us so i switched to lybrel because the ingredients are similar when my other pills ended i started lybrel immediately on my first day of period as the instructions said and the period lasted for two weeks when taking the second pack same two weeks and now with third pack things got even worse my third period lasted for two weeks and now its the end of the third week i still have daily brown discharge\nthe positive side is that i didnt have any other side effects the idea of being period free was so tempting alas,2.203,-1.215,0.955,-0.738,-3.010,-1.069
this is my first time using any form of birth control im glad i went with the patch i have been on it for months at first it decreased my libido but that subsided the only downside is that it made my periods longer days to be exact i used to only have periods for days max also made my cramps intense for the first two days of my period i never had cramps before using birth control other than that in happy with the patch,1.881,-2.050,2.058,-1.216,1.105,-0.344
suboxone has completely turned my life around i feel healthier im excelling at my job and i always have money in my pocket and my savings account i had none of those before suboxone and spent years abusing oxycontin my paycheck was already spent by the time i got it and i started resorting to scheming and stealing to fund my addiction all that is history if youre ready to stop theres a good chance that suboxone will put you on the path of great life again i have found the side effects to be minimal compared to oxycontin im actually sleeping better slight constipation is about it for me it truly is amazing the cost pales in comparison to what i spent on oxycontin,0.769,0.031,-0.435,-0.059,-0.001,0.146
...,...,...,...,...,...,...
i have taken tamoxifen for years side effects are severe sweating and depression i have been taking effexor xr longer than i have been on tamoxifen my oncologist increased the effexor dosage from mg to mg per day she assure me the effexor and black cohoosh would stop the sweatingnot sweating increased and i am more depressed than ever i had a sonogram last month that revealed a very small fibroid and fluid in my uterus got an appointment with gyn next week to see how she wants to handle the uterus problem,0.710,0.072,-0.383,-0.122,0.056,-0.177
ive been taking lexapro escitaploprgram since february first id like to mention that you can not take this drug for a week or less and expect to magically feel better i felt really sick the first two weeks on this drug but you have to give the drug time for me i didnt really start noticing the drugs positive effects for about two months i took zoloft before this and felt like it made me too tired and absent minded luckily lexapro doesnt seem to have this effect although i do drink caffeinated drinks i like lexapro not only because my anxiety and depression is completely

In [16]:
cosine_similarity((doc_topic[0], doc_topic[1])).round()

array([[1., 1.],
       [1., 1.]])

In [17]:
cosine_similarity((doc_topic[0], doc_topic[6])).round()

array([[1., 0.],
       [0., 1.]])

In [18]:
Vt.head()

,period,pain,birth control,weight,mood,acne
review,,,,,,
it has no side effect i take it in combination of bystolic mg and fish oil,0.053,0.009,-0.041,0.006,0.012,0.000
my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective,1.810,0.084,-0.810,-0.278,-0.291,0.385
i used to take another oral contraceptive which had pill cycle and was very happy very light periods max days no other side effects but it contained hormone gestodene which is not available in us so i switched to lybrel because the ingredients are similar when my other pills ended i started lybrel immediately on my first day of period as the instructions said and the period lasted for two weeks when taking the second pack same two weeks and now with third pack things got even worse my third period lasted for two weeks and now its the end of the third week i still have daily brown discharge\nthe positive side is that i didnt have any other side effects the idea of being period free was so tempting alas,2.203,-1.215,0.955,-0.738,-3.010,-1.069
this is my first time using any form of birth control im glad i went with the patch i have been on it for months at first it decreased my libido but that subsided the only downside is that it made my periods longer days to be exact i used to only have periods for days max also made my cramps intense for the first two days of my period i never had cramps before using birth control other than that in happy with the patch,1.881,-2.050,2.058,-1.216,1.105,-0.344
suboxone has completely turned my life around i feel healthier im excelling at my job and i always have money in my pocket and my savings account i had none of those before suboxone and spent years abusing oxycontin my paycheck was already spent by the time i got it and i started resorting to scheming and stealing to fund my addiction all that is history if youre ready to stop theres a good chance that suboxone will put you on the path of great life again i have found the side effects to be minimal compared to oxycontin im actually sleeping better slight constipation is about it for me it truly is amazing the cost pales in comparison to what i spent on oxycontin,0.769,0.031,-0.435,-0.059,-0.001,0.146


# Recommendation System - Content Based Filtering

In [19]:
Vt.head()

,period,pain,birth control,weight,mood,acne
review,,,,,,
it has no side effect i take it in combination of bystolic mg and fish oil,0.053,0.009,-0.041,0.006,0.012,0.000
my son is halfway through his fourth week of intuniv we became concerned when he began this last week when he started taking the highest dose he will be on for two days he could hardly get out of bed was very cranky and slept for nearly hours on a drive home from school vacation very unusual for him i called his doctor on monday morning and she said to stick it out a few days see how he did at school and with getting up in the morning the last two days have been problem free he is much more agreeable than ever he is less emotional a good thing less cranky he is remembering all the things he should overall his behavior is better we have tried many different medications and so far this is the most effective,1.810,0.084,-0.810,-0.278,-0.291,0.385
i used to take another oral contraceptive which had pill cycle and was very happy very light periods max days no other side effects but it contained hormone gestodene which is not available in us so i switched to lybrel because the ingredients are similar when my other pills ended i started lybrel immediately on my first day of period as the instructions said and the period lasted for two weeks when taking the second pack same two weeks and now with third pack things got even worse my third period lasted for two weeks and now its the end of the third week i still have daily brown discharge\nthe positive side is that i didnt have any other side effects the idea of being period free was so tempting alas,2.203,-1.215,0.955,-0.738,-3.010,-1.069
this is my first time using any form of birth control im glad i went with the patch i have been on it for months at first it decreased my libido but that subsided the only downside is that it made my periods longer days to be exact i used to only have periods for days max also made my cramps intense for the first two days of my period i never had cramps before using birth control other than that in happy with the patch,1.881,-2.050,2.058,-1.216,1.105,-0.344
suboxone has completely turned my life around i feel healthier im excelling at my job and i always have money in my pocket and my savings account i had none of those before suboxone and spent years abusing oxycontin my paycheck was already spent by the time i got it and i started resorting to scheming and stealing to fund my addiction all that is history if youre ready to stop theres a good chance that suboxone will put you on the path of great life again i have found the side effects to be minimal compared to oxycontin im actually sleeping better slight constipation is about it for me it truly is amazing the cost pales in comparison to what i spent on oxycontin,0.769,0.031,-0.435,-0.059,-0.001,0.146


In [20]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212850 entries, 0 to 212849
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   drugName     212850 non-null  object 
 1   condition    212850 non-null  object 
 2   review       212850 non-null  object 
 3   rating       212850 non-null  float64
 4   date         212850 non-null  object 
 5   usefulCount  212850 non-null  int64  
 6   strlen       212850 non-null  int64  
 7   tokens       212850 non-null  object 
 8   text         212850 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 14.6+ MB


In [21]:
df_clean.nunique()

drugName         3665
condition         916
review         128776
rating             10
date             3579
usefulCount       396
strlen            180
tokens         127907
text           128776
dtype: int64

In [22]:
# making medical condition as index column

Vt2 = pd.DataFrame(doc_topic.round(3),
             index = df_clean.condition,
             columns = ['period', 'pain', 'birth control', 'weight', 'mood', 'acne'])
Vt2 = Vt2.sample(10000)

In [23]:
# adding drugName column for easy identification

drugName_df = df_clean[['drugName', 'condition']]
drug_df = drugName_df.sample(10000)

In [24]:
# merging two dfs

rec_df = pd.merge(Vt2, drug_df, on=['condition'])
rec_df

,condition,period,pain,birth control,weight,mood,acne,drugName
0,Hot Flashes,1.827,-1.057,-0.141,2.773,0.700,0.154,Venlafaxine
1,Hot Flashes,1.827,-1.057,-0.141,2.773,0.700,0.154,Paroxetine
2,Hot Flashes,1.827,-1.057,-0.141,2.773,0.700,0.154,Paroxetine
3,Hot Flashes,1.827,-1.057,-0.141,2.773,0.700,0.154,Gabapentin
4,Hot Flashes,1.827,-1.057,-0.141,2.773,0.700,0.154,Venlafaxine
...,...,...,...,...,...,...,...,...
4662086,"Anemia, Sickle Cell",0.501,0.038,-0.302,-0.002,-0.004,0.056,Hydroxyurea
4662087,Neck Pain,0.101,0.004,-0.039,-0.019,-0.004,0.030,Ibuprofen
4662088,Prevention of Dental Caries,0.741,0.019,-0.319,-0.020,-0.198,0.275,Fluoride
4662089,16</span> users found this comment helpful.,0.859,-0.193,-0.123,-0.260,0.458,0.167,Lyrica


In [25]:
rec_df.nunique()

condition         364
period           2511
pain             2706
birth control    2816
weight           2285
mood             2236
acne             2121
drugName         1400
dtype: int64

In [26]:
rec_df.condition.value_counts()

Birth Control                                  3360000
Depression                                      349782
Pain                                            129519
Anxiety                                         126732
Acne                                            112450
                                                ...   
Dumping Syndrome                                     1
Post-Cholecystectomy Diarrhea                        1
Lyme Disease                                         1
85</span> users found this comment helpful.          1
Nasal Polyps                                         1
Name: condition, Length: 364, dtype: int64

In [28]:
rec_df.condition.value_counts().head(6)

Birth Control      3360000
Depression          349782
Pain                129519
Anxiety             126732
Acne                112450
Bipolar Disorde      72474
Name: condition, dtype: int64

In [29]:
rec_df.to_csv("C:/Users/New User/Desktop/NLP/drugsCom_raw/st_data.csv")

In [252]:
rec_df = pd.read_csv("C:/Users/New User/Desktop/NLP/drugsCom_raw/st_data.csv")

In [253]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4662091 entries, 0 to 4662090
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   index          int64  
 2   condition      object 
 3   period         float64
 4   pain           float64
 5   birth control  float64
 6   weight         float64
 7   mood           float64
 8   acne           float64
 9   drugName       object 
dtypes: float64(6), int64(2), object(2)
memory usage: 355.7+ MB


In [269]:
rec_df_10k = rec_df.sample(10000)
rec_df_10k = rec_df_10k.drop(['index'], axis=1).reset_index()
rec_df_10k =  rec_df_10k.drop(['Unnamed: 0'], axis=1).reset_index()
rec_df_10k = rec_df_10k.drop(['level_0'], axis=1)
rec_df_10k = rec_df_10k.drop(['index'], axis=1).reset_index()
rec_df_10k

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,Depression,0.802,0.159,-0.614,-0.089,-0.026,0.084,Wellbutrin SR
1,1,Weight Loss,2.178,0.023,-1.142,0.323,-0.463,0.498,Phentermine
2,2,Birth Control,1.044,-0.399,0.289,-0.353,-1.159,-0.340,Nexplanon
3,3,Birth Control,0.606,-0.266,-0.004,0.455,-0.110,0.511,Ethinyl estradiol / norethindrone
4,4,Birth Control,1.734,0.358,0.022,1.245,0.274,-0.582,Junel Fe 1 / 20
...,...,...,...,...,...,...,...,...,...
9995,9995,Birth Control,1.923,1.159,1.057,0.107,-0.422,0.313,Ortho Evra
9996,9996,Birth Control,0.992,-0.083,-0.202,-0.246,-0.381,0.040,Etonogestrel
9997,9997,Birth Control,1.873,0.126,0.505,0.515,-0.889,-0.413,Ethinyl estradiol / norethindrone
9998,9998,Birth Control,2.623,3.014,1.862,0.113,-0.214,-0.108,Sprintec


In [200]:
rec_df = rec_df_10.drop(['index'], axis=1).reset_index()
rec_df

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
1,1,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
2,2,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3,3,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
4,4,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
5,5,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
6,6,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
7,7,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
8,8,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
9,9,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [157]:
rec_df = rec_df.reset_index()

In [136]:
rec_df.tail()

,Unnamed: 0,index,condition,period,pain,birth control,weight,mood,acne,drugName
4662086,4662086,4662086,"Anemia, Sickle Cell",0.501,0.038,-0.302,-0.002,-0.004,0.056,Hydroxyurea
4662087,4662087,4662087,Neck Pain,0.101,0.004,-0.039,-0.019,-0.004,0.030,Ibuprofen
4662088,4662088,4662088,Prevention of Dental Caries,0.741,0.019,-0.319,-0.020,-0.198,0.275,Fluoride
4662089,4662089,4662089,16</span> users found this comment helpful.,0.859,-0.193,-0.123,-0.260,0.458,0.167,Lyrica
4662090,4662090,4662090,16</span> users found this comment helpful.,0.859,-0.193,-0.123,-0.260,0.458,0.167,Gralise


In [158]:
rec_df = rec_df.drop(['Unnamed: 0', 'level_0'], axis=1)
rec_df.head()

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Venlafaxine
1,1,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Paroxetine
2,2,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Paroxetine
3,3,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Gabapentin
4,4,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Venlafaxine


In [138]:
rec_df.to_csv("C:/Users/New User/Desktop/NLP/drugsCom_raw/st_data.csv")

In [152]:
rec_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4662091 entries, 0 to 4662090
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   level_0        int64  
 1   index          int64  
 2   condition      object 
 3   period         float64
 4   pain           float64
 5   birth control  float64
 6   weight         float64
 7   mood           float64
 8   acne           float64
 9   drugName       object 
dtypes: float64(6), int64(2), object(2)
memory usage: 355.7+ MB


In [145]:
rec_df = rec_df.reset_index()
rec_df.head()

,level_0,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,0,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Venlafaxine
1,1,1,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Paroxetine
2,2,2,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Paroxetine
3,3,3,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Gabapentin
4,4,4,Hot Flashes,1.827,-1.057,-0.141,2.773,0.7,0.154,Venlafaxine


In [151]:
rec_df_sample = rec_df.sample(10000)
rec_df_sample.head()

,level_0,index,condition,period,pain,birth control,weight,mood,acne,drugName
592667,592667,592667,Birth Control,0.374,0.002,-0.154,-0.041,-0.097,0.048,Implanon
1283072,1283072,1283072,Birth Control,2.441,-0.906,0.254,0.603,-1.087,-0.085,Microgestin Fe 1 / 20
1857512,1857512,1857512,Birth Control,1.771,-1.332,1.329,-0.896,-0.360,-0.513,Skyla
2264914,2264914,2264914,Birth Control,2.337,-1.362,0.872,-0.720,-0.189,0.317,Etonogestrel
3864432,3864432,3864432,Weight Loss,0.285,-0.031,-0.122,-0.071,0.021,0.012,Lorcaserin


In [ ]:
rec_df_sample.reset_index()
rec_df_sample

In [166]:
rec_df_10 = rec_df.sample(10)
rec_df_10

,index,condition,period,pain,birth control,weight,mood,acne,drugName
3765949,3765949,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
3417382,3417382,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
1774721,1774721,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3823952,3823952,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
3480146,3480146,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
4498577,4498577,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
1180543,1180543,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
1641435,1641435,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
2998135,2998135,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
831186,831186,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [167]:
rec_df_10 = rec_df_10.reset_index(drop=True)
rec_df_10

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,3765949,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
1,3417382,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
2,1774721,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3,3823952,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
4,3480146,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
5,4498577,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
6,1180543,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
7,1641435,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
8,2998135,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
9,831186,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [168]:
rec_df_10 = rec_df_10.drop(['index'], axis=1)
rec_df_10

,condition,period,pain,birth control,weight,mood,acne,drugName
0,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
1,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
2,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
4,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
5,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
6,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
7,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
8,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
9,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [169]:
rec_df_10 = rec_df_10.reset_index()
rec_df_10

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
1,1,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
2,2,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3,3,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
4,4,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
5,5,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
6,6,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
7,7,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
8,8,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
9,9,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [171]:
rec_df_100 = rec_df.sample(100)
rec_df_100 = rec_df_10.drop(['index'], axis=1).reset_index()
rec_df_100

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,0,Bipolar Disorde,0.527,0.127,-0.589,0.288,0.013,0.199,Quetiapine
1,1,Birth Control,1.969,-0.991,0.197,0.379,-0.928,0.344,Sprintec
2,2,Birth Control,1.886,-1.836,1.590,0.604,-0.343,-0.859,Mirena
3,3,Insomnia,1.240,0.607,0.241,-0.050,0.521,-0.020,Gabapentin
4,4,Birth Control,1.867,-1.050,0.007,0.839,0.117,1.310,Skyla
5,5,Obesity,0.563,-0.076,-0.395,0.605,0.018,0.205,Saxenda
6,6,Birth Control,1.807,-0.219,-0.615,0.474,-0.107,0.536,Junel Fe 1 / 20
7,7,Birth Control,1.318,0.421,0.727,-0.238,-0.929,-0.370,Implanon
8,8,Birth Control,2.145,-0.277,0.829,1.273,-0.621,-0.646,Ethinyl estradiol / levonorgestrel
9,9,Birth Control,1.541,-0.660,-0.033,0.372,-0.494,-0.186,Drospirenone / ethinyl estradiol


In [280]:
# DRUG RECOMMENDATION PROGRAM

def drug_rec(user_condition):
    def get_index_from_condition(condition):
        return rec_df_10k[rec_df_10k.condition == condition]['index'].values[0]

    def get_drug_from_index(index):
        return rec_df_10k[rec_df_10k.index == index]['drugName'].values[0]

    cv=CountVectorizer()
    count_matrix=cv.fit_transform(rec_df_10k)
    cosine_sim=cosine_similarity(count_matrix)
    
    condition_index = get_index_from_condition(user_condition)
    similar_drugs=list(enumerate(cosine_sim[condition_index]))
    sorted_similar_drugs = sorted(similar_drugs, key=lambda x: x[1], reverse=True)
    i=0
    for drug in sorted_similar_drugs:
        print (get_drug_from_index(drug[0]))
        i=i+1
        if i>5:
            break
    return get_drug_from_index(drug[0])

user_condition = "Birth Control"
drug_rec(user_condition)

Nexplanon
Wellbutrin SR
Phentermine
Ethinyl estradiol / norethindrone
Junel Fe 1 / 20
Baclofen


'Baclofen'

In [237]:
# DRUG RECOMMENDATION PROGRAM

def drug_rec(user_condition):
    def get_index_from_condition(condition):
        return rec_df[rec_df.condition == condition]['index'].values[0]

    def get_drug_from_index(index):
        return rec_df[rec_df.index == index]['drugName'].values[0]

    cv=CountVectorizer()
    count_matrix=cv.fit_transform(rec_df)
    cosine_sim=cosine_similarity(count_matrix)
    
    condition_index = get_index_from_condition(user_condition)
    similar_drugs=list(enumerate(cosine_sim[condition_index]))
    sorted_similar_drugs = sorted(similar_drugs, key=lambda x: x[1], reverse=True)
    i=0
    for drug in sorted_similar_drugs:
        print (get_drug_from_index(drug[0]))
        i=i+1
        if i>5:
            break
    return get_drug_from_index(drug[0])

In [250]:
user_condition = "Birth Control"
drug_rec(user_condition)

Sprintec
Quetiapine
Mirena
Gabapentin
Skyla
Saxenda


'Saxenda'

In [221]:
# Helper functions

def get_index_from_condition(condition):
    return rec_df[rec_df.condition == condition]['index'].values[0]

def get_drug_from_index(index):
    return rec_df[rec_df.index == index]['drugName'].values[0]




In [222]:
cv=CountVectorizer()
count_matrix=cv.fit_transform(rec_df)


cosine_sim=cosine_similarity(count_matrix)

In [228]:
user_condition = "Birth Control"

In [229]:
condition_index = get_index_from_condition(user_condition)
print(condition_index)

1


In [230]:
similar_drugs=list(enumerate(cosine_sim[condition_index]))

In [231]:
sorted_similar_drugs = sorted(similar_drugs, key=lambda x: x[1], reverse=True)

In [232]:
i=0
for drug in sorted_similar_drugs:
    print (get_drug_from_index(drug[0]))
    i=i+1
    if i>5:
        break

Sprintec
Quetiapine
Mirena
Gabapentin
Skyla
Saxenda


In [32]:
rec_df_sample.to_csv("C:/Users/New User/Desktop/NLP/drugsCom_raw/st_data_sample.csv")

In [33]:
# Applying Principal Component Analysis (PCA) on feature matrix to reduce down to 3 features

from sklearn.decomposition import PCA

pca = PCA(n_components = 3)
new_coords = pca.fit_transform(rec_df_sample[['period', 'pain', 'birth control', 'weight', 'mood', 'acne']])

In [34]:
pd.DataFrame(new_coords, index=rec_df_sample.condition)

,0,1,2
condition,,,
Bipolar Disorde,-0.352738,-0.109404,2.226629
Depression,-0.987548,-0.564593,-0.086458
Birth Control,1.535994,0.277875,1.181279
Birth Control,-0.586854,-0.584910,-0.149616
Birth Control,-0.790739,0.271313,-0.279445
...,...,...,...
Anxiety,-1.137429,-0.567683,-0.183720
Acne,-0.769201,-0.618913,-0.184567
Birth Control,1.080095,-0.056396,0.497591


In [46]:
import streamlit as st

st.title("PeopleRX: A Drug Recommendation System For People By The People")

st.header("This system recommends top 3 most effective medications for a chosen medical condition")

st.subheader("Enter a medical condition:")

st.selectbox("Condition", ['cold', 'headache', 'acne'], index=1)

if st.button("Recommend Me:"):
    st.write("The Top 3 Suggested Medications Are:")





st.text("Disclaimer: \nThis system and its recommendations are not official medical advice. \nPlease use at your own risk.")

2022-04-18 22:55:27.851 
  command:

    streamlit run C:\Users\New User\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [85]:
from sklearn.metrics import pairwise_distances

dist = pairwise_distances(new_coords[0].reshape(1, -1), new_coords)

candidate_list = dist.argsort().tolist()
print(candidate_list)

[[0, 251, 7908, 9282, 517, 893, 8733, 9352, 1481, 9219, 6746, 6828, 309, 1796, 3441, 8171, 9201, 5940, 8036, 2634, 8093, 8593, 7788, 8434, 9355, 6616, 8340, 3697, 4730, 1401, 4913, 4505, 2730, 8400, 6615, 1428, 7886, 1127, 8962, 9557, 8927, 6420, 8275, 4754, 8627, 1912, 5453, 6016, 966, 2006, 1635, 980, 6832, 6206, 3423, 3999, 218, 7101, 2275, 4652, 5225, 7924, 3826, 4054, 6946, 3686, 7907, 5066, 589, 5029, 3713, 5470, 6032, 94, 3840, 8317, 7053, 3104, 215, 653, 5757, 3756, 4899, 4785, 4291, 9902, 7593, 5551, 1930, 9631, 8238, 6357, 3355, 9802, 9082, 3573, 9841, 885, 9403, 7537, 716, 4858, 9326, 9564, 9044, 6830, 3147, 1478, 860, 2405, 4012, 5898, 9678, 2750, 5031, 1149, 911, 8815, 6231, 2386, 9907, 8867, 7973, 3670, 7020, 6928, 3263, 9687, 1295, 398, 9184, 6676, 8654, 5853, 3271, 564, 5063, 5036, 8640, 9694, 2441, 2246, 1347, 8471, 5742, 6664, 3939, 7081, 1651, 2827, 8236, 1554, 2872, 3256, 6300, 2640, 2570, 1794, 7819, 7190, 8452, 2644, 5644, 8683, 8455, 240, 211, 8591, 254, 2530, 49

In [81]:
top_3_meds = candidate_list[0][1:4]

In [82]:
top_3_meds

[251, 7908, 9282]

In [94]:
rec_df_sample.iloc[[top_3_meds][0]]

,condition,period,pain,birth control,weight,mood,acne,drugName
4334398,Depression,2.211,-0.45,-1.028,2.030,0.352,0.002,Celexa
4334651,Depression,2.211,-0.45,-1.028,2.030,0.352,0.002,Celexa
3068497,Birth Control,1.535,-0.68,-0.461,2.015,0.247,-0.108,Etonogestrel


In [118]:
# writing a top-3-drug-recommendation program

def drug_rec(condition):
    top_meds = []
    if condition in rec_df_sample.condition:
        dist = pairwise_distances(condition[0].reshape(1, -1), new_coords)
        print(dist)
#         top_meds.append(dist.argsort())
#     return top_meds

drug_rec(condition)

In [119]:
rec_df_sample.head()

,condition,period,pain,birth control,weight,mood,acne,drugName
3720856,Bipolar Disorde,1.678,-0.632,-0.563,2.136,0.206,0.034,Oxcarbazepine
4392906,Depression,0.875,0.005,-0.739,-0.199,0.133,-0.408,Aripiprazole
452118,Birth Control,2.624,-1.694,0.595,0.931,0.482,-0.046,Liletta
1063412,Birth Control,1.117,-0.185,-0.317,-0.298,-0.036,0.875,Nexplanon
174197,Birth Control,0.767,-0.257,0.014,-0.202,-0.640,-0.308,Drospirenone / ethinyl estradiol


In [120]:
rec_df_sample = rec_df_sample.reset_index()
rec_df_sample.head()

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,3720856,Bipolar Disorde,1.678,-0.632,-0.563,2.136,0.206,0.034,Oxcarbazepine
1,4392906,Depression,0.875,0.005,-0.739,-0.199,0.133,-0.408,Aripiprazole
2,452118,Birth Control,2.624,-1.694,0.595,0.931,0.482,-0.046,Liletta
3,1063412,Birth Control,1.117,-0.185,-0.317,-0.298,-0.036,0.875,Nexplanon
4,174197,Birth Control,0.767,-0.257,0.014,-0.202,-0.640,-0.308,Drospirenone / ethinyl estradiol


In [121]:
rec_df_sample.iloc[[0]]

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,3720856,Bipolar Disorde,1.678,-0.632,-0.563,2.136,0.206,0.034,Oxcarbazepine


In [132]:
# Helper functions

def get_index_from_condition(condition):
    return rec_df_sample[rec_df_sample.condition == condition]['index'].values[0]

# def get_drug_from_index(index):
#     return rec_df_sample[rec_df_sample.index == index]['drugName'].values[0]

# def get_index_from_drug(drug):
#     return rec_df_sample[rec_df_sample.drugName == drug]['index'].values[0]



In [133]:
get_index_from_condition("Depression")

4392906

In [140]:
rec_df_sample.head()

,index,condition,period,pain,birth control,weight,mood,acne,drugName
0,3720856,Bipolar Disorde,1.678,-0.632,-0.563,2.136,0.206,0.034,Oxcarbazepine
1,4392906,Depression,0.875,0.005,-0.739,-0.199,0.133,-0.408,Aripiprazole
2,452118,Birth Control,2.624,-1.694,0.595,0.931,0.482,-0.046,Liletta
3,1063412,Birth Control,1.117,-0.185,-0.317,-0.298,-0.036,0.875,Nexplanon
4,174197,Birth Control,0.767,-0.257,0.014,-0.202,-0.640,-0.308,Drospirenone / ethinyl estradiol


In [144]:
rec_df_sample